In [18]:
dataset_name = "jsm-test-dataset" # Name of the Voxel51 Dataset to use

### Load the Voxel51 Dataset

In [2]:
#config
import fiftyone as fo
from fiftyone import ViewField as F
import os
import numpy as np
import matplotlib.pyplot as plt


dataset = fo.load_dataset(dataset_name)  

### Browse the entire Voxel51 Dataset

In [3]:
session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


### Browse a subsection of the Voxel51 Dataset

In [24]:
#view = dataset.exists("training")

# Veiw samples where there is no label
# view = dataset.match({"model": {"$exists": False, "$eq": None}})

# View samples with a tag
view = dataset.match_tags("eval")

print(view)

session.view = view


Dataset:     jsm-test-dataset
Media type:  image
Num samples: 485
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'eval']
Sample fields:
    id:                   fiftyone.core.fields.ObjectIdField
    filepath:             fiftyone.core.fields.StringField
    tags:                 fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    bearing:              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    elevation:            fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    distance:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:               fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    mo

### Add Tags to Selected Samples
Use the browser to select samples and add tags to them

In [8]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)
print(selected_view)
for sample in selected_view:
    sample.tags.append("training")
    sample.save() 

Dataset:     jsm-test-dataset
Media type:  image
Num samples: 2
Tags:        []
Sample fields:
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    bearing:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    elevation:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    distance:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    model:            fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    manufacturer:     fiftyone.core.fields.EmbeddedDo

### Remove Tags to Selected Samples
Use the browser to select samples and remove tags from them

In [23]:
# Create a view containing only the selected samples
selected_view = dataset.select(session.selected)
print(selected_view)
for sample in selected_view:
    sample.tags.remove("eval")
    sample.save() 

Dataset:     jsm-test-dataset
Media type:  image
Num samples: 29
Tags:        ['capture-3-29', 'capture-3-30', 'capture-5-13', 'training']
Sample fields:
    id:                   fiftyone.core.fields.ObjectIdField
    filepath:             fiftyone.core.fields.StringField
    tags:                 fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    external_id:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    bearing:              fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    elevation:            fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    distance:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    icao24:               fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
   

ValueError: list.remove(x): x not in list

### Add a Tag to All Samples with a given Label Field

In [ ]:
view = dataset.exists("detections")
print(view)
for sample in view:
    sample.tags.append("training")
    sample.save() 

### Add a Tag to All Samples in a Dataset

In [10]:
for sample in dataset:
    sample.tags.append("capture-5-13")
    sample.save() 

### Number of Unique Aircraft Bodies per Normalized Model

In [27]:

norm_models = dataset.distinct("norm_model.label")
for norm_model in norm_models:
    view = dataset.filter_labels("norm_model", (F("label") == norm_model)).select_fields("icao24")
    unique_aircraft = view.distinct("icao24.label")
    print("{}: {}".format(norm_model,len(unique_aircraft)))


737-700: 8
737-800: 24
737-900: 11
757-200: 1
777-200: 2
787-800: 2
A319: 12
A320: 71
A321: 25
Bombardier Challanger: 8
CRJ550: 2
CRJ700: 5
CRJ900: 9
Cessna Jet: 14
EMB-135: 8
EMB-505: 5
ERJ-170: 62
Falcon: 1
Gulfstream: 6
Hawker: 2
King Air: 1
Learjet 45/60: 3
Pilatus PC-12: 2

Session disconnected, trying to reconnect



Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/fiftyone/core/client.py", line 122, in run_client
    io_loop.run_sync(connect)
  File "/usr/local/lib/python3.6/dist-packages/tornado/ioloop.py", line 530, in run_sync
    return future_cell[0].result()
  File "/usr/local/lib/python3.6/dist-packages/fiftyone/core/client.py", line 89, in connect
    message = json_util.loads(message)
  File "/usr/local/lib/python3.6/dist-packages/bson/json_util.py", line 439, in loads
    return json.loads(s, *args, **kwargs)
  File "/usr/lib/python3.6/json/__init__.py", line 348, in loads
    'not {!r}'.format(s.__class__.__name__))
TypeError: the JSON object must be str, bytes or bytearray, not 'NoneType'
Exception in thread Thread-5:
Traceback (most r

### Add Detection Evaluation

In [52]:
view = dataset.match_tags("eval")
for sample in view:
    if sample["detections"] == None:
        sample["detections"] = fo.Detections(detections=[])
        sample.save()

<SampleView: {
    'id': '60a3bf27f3610f2b7a82541b',
    'media_type': 'image',
    'filepath': '/tf/media/capture-5-13/Boeing 737-832/a4417d_193_54_14213_2021-05-13-12-02-17.jpg',
    'tags': BaseList(['capture-5-13', 'eval']),
    'metadata': <ImageMetadata: {
        'size_bytes': 892032,
        'mime_type': 'image/jpeg',
        'width': 1920,
        'height': 1080,
        'num_channels': 3,
    }>,
    'external_id': <Classification: {
        'id': '60a3bf27f3610f2b7a825416',
        'tags': BaseList([]),
        'label': 'a4417d_193_54_14213_2021-05-13-12-02-17',
        'confidence': None,
        'logits': None,
    }>,
    'bearing': <Classification: {
        'id': '60a3bf27f3610f2b7a825417',
        'tags': BaseList([]),
        'label': '193',
        'confidence': None,
        'logits': None,
    }>,
    'elevation': <Classification: {
        'id': '60a3bf27f3610f2b7a825418',
        'tags': BaseList([]),
        'label': '54',
        'confidence': None,
        'lo

In [51]:
# Evaluate `predictions` w.r.t. labels in `ground_truth` field
view = dataset.match_tags("eval")
print(view.first())
view.evaluate_detections(
    "881_40k_predict", gt_field="detections", eval_key="eval_881"
)



<SampleView: {
    'id': '60a3bf27f3610f2b7a82541b',
    'media_type': 'image',
    'filepath': '/tf/media/capture-5-13/Boeing 737-832/a4417d_193_54_14213_2021-05-13-12-02-17.jpg',
    'tags': BaseList(['capture-5-13', 'eval']),
    'metadata': <ImageMetadata: {
        'size_bytes': 892032,
        'mime_type': 'image/jpeg',
        'width': 1920,
        'height': 1080,
        'num_channels': 3,
    }>,
    'external_id': <Classification: {
        'id': '60a3bf27f3610f2b7a825416',
        'tags': BaseList([]),
        'label': 'a4417d_193_54_14213_2021-05-13-12-02-17',
        'confidence': None,
        'logits': None,
    }>,
    'bearing': <Classification: {
        'id': '60a3bf27f3610f2b7a825417',
        'tags': BaseList([]),
        'label': '193',
        'confidence': None,
        'logits': None,
    }>,
    'elevation': <Classification: {
        'id': '60a3bf27f3610f2b7a825418',
        'tags': BaseList([]),
        'label': '54',
        'confidence': None,
        'lo

AttributeError: 'NoneType' object has no attribute 'detections'

In [ ]:
# Evaluate the detections in the `predictions` field with respect to the
# objects in the `ground_truth` field
results = view.evaluate_detections(
    "881_40k_predict", gt_field="detections", eval_key="eval_881", compute_mAP=True
)

# Get the 10 most common classes in the dataset
counts = view.count_values("detections.detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
results.print_report(classes=classes)

# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

# Create a view that has samples with the most false positives first, and
# only includes false positive boxes in the `predictions` field
eval_view = (
    view
    .sort_by("eval_fp", reverse=True)
    .filter_labels("914_40k_predict", F("eval") == "fp")
)
print("mAP: {}".format(results.mAP()))
# 0.3957

plot = results.plot_pr_curves(classes=["plane"])
plot.show()

# Visualize results in the App
session = fo.launch_app(view=eval_view, auto=False)

In [ ]:
session = fo.launch_app(dataset, auto=False)

# Convert to evaluation patches
eval_patches = view.to_evaluation_patches("eval")
print(eval_patches)

# View patches in the App
session.view = eval_patches